# Spark Setup and Data Load

##Installation of Spark

In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark2.4.5
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
# unzip it
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
# install findspark
!pip install -q findspark
# Google Colab has Java 11 available, test it using below command -
!ls /usr/lib/jvm
#install pyarrow
!pip install -U pyarrow
# clone github repo
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-films

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 35.4 MB 396 kB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 10.0.0 which is incompatible.
db-dtypes 1.0.4 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 10.0.0 which is incompatible.
Cloning into 'BigData2022-films'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 213

In [ ]:
import os
import findspark

Now we need to setup environment variables for our Spark Session to work

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

To verify the automatically detected location we use findspark.init()

In [ ]:
findspark.init()
from pyspark.sql import SparkSession

Time to initiate a Spark Session

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark_params = {
"spark.executor.memory" : "4g",
"spark.driver.memory": "4g",
"spark.memory.fraction": "0.9"}
for param, value in spark_params.items():
  spark.conf.set(param, value)

In [ ]:
spark

## Load dataset

In [ ]:
!wget "https://datasets.imdbws.com/name.basics.tsv.gz"
!wget "https://datasets.imdbws.com/title.akas.tsv.gz"
!wget "https://datasets.imdbws.com/title.basics.tsv.gz"
!wget "https://datasets.imdbws.com/title.crew.tsv.gz"
!wget "https://datasets.imdbws.com/title.episode.tsv.gz"
!wget "https://datasets.imdbws.com/title.principals.tsv.gz"
!wget "https://datasets.imdbws.com/title.ratings.tsv.gz"

--2022-11-05 16:53:52--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.41, 99.84.160.101, 99.84.160.48, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235998305 (225M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.07M   105MB/s    in 2.1s    

2022-11-05 16:53:55 (105 MB/s) - ‘name.basics.tsv.gz’ saved [235998305/235998305]

--2022-11-05 16:53:55--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.41, 99.84.160.101, 99.84.160.48, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286667472 (273M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 273.39M  37.9

In [ ]:
title_ratings = spark.read.csv("title.ratings.tsv.gz", sep='\t', header=True)
title_principals = spark.read.csv("title.principals.tsv.gz", sep='\t',header=True)
title_episode = spark.read.csv("title.episode.tsv.gz", sep='\t', header=True)
title_crew = spark.read.csv("title.crew.tsv.gz", sep='\t', header=True)
title_basics = spark.read.csv("title.basics.tsv.gz", sep='\t', header=True)
title_akas = spark.read.csv("title.akas.tsv.gz", sep='\t', header=True)
name_basics = spark.read.csv("name.basics.tsv.gz", sep='\t', header=True)

In [ ]:
title_ratings.show(5)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1923|
|tt0000002|          5.8|     260|
|tt0000003|          6.5|    1733|
|tt0000004|          5.6|     174|
|tt0000005|          6.2|    2545|
+---------+-------------+--------+
only showing top 5 rows



# Prepare Data

Data overview

In [ ]:
print("title_ratings")
title_ratings.show(2)
print("title_principals")
title_principals.show(2)
print("title_episode")
title_episode.show(2)
print("title_crew")
title_crew.show(2)
print("title_basics")
title_basics.show(2)
print("title_akas")
title_akas.show(2)
print("name_basics")
name_basics.show(2)

title_ratings
+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1923|
|tt0000002|          5.8|     260|
+---------+-------------+--------+
only showing top 2 rows

title_principals
+---------+--------+---------+--------+---+----------+
|   tconst|ordering|   nconst|category|job|characters|
+---------+--------+---------+--------+---+----------+
|tt0000001|       1|nm1588970|    self| \N|  ["Self"]|
|tt0000001|       2|nm0005690|director| \N|        \N|
+---------+--------+---------+--------+---+----------+
only showing top 2 rows

title_episode
+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
+---------+------------+------------+-------------+
only showing top 2 rows

title_crew
+---------+------

Join tables

In [ ]:
temp_akas = title_akas.filter(title_akas.isOriginalTitle == 1)
temp_akas = temp_akas.select(["titleId", "region", "language"]).distinct()
temp_akas = temp_akas.withColumnRenamed("titleId", "tconst")

to_print = ["title_basics", "title_ratings", "title_principals", 
            "title_episode", "name_basics", "temp_akas"]

for p in to_print:
  print(f"Dimension {p}: ({eval(p).count()}, {len(eval(p).columns)})")

data = title_basics.join(title_ratings, how="left", on="tconst")
print(f"\nJoined title_principals to title_basics\n" + 
      f"Dimension: ({data.count()}, {len(data.columns)})")

data = data.join(title_principals, how="left", on="tconst")
print(f"Joined title_principals\n" + 
      f"Dimension: ({data.count()}, {len(data.columns)})")

data = data.join(title_episode, how="left", on="tconst")
print(f"Joined title_episode\n" + 
      f"Dimension: ({data.count()}, {len(data.columns)})")

data = data.join(name_basics, how="left", on="nconst")
print(f"Joined name_basics\n" + 
      f"Dimension: ({data.count()}, {len(data.columns)})")

data = data.join(temp_akas, how="left", on="tconst")
print(f"Joined temp_akas\n" + 
      f"Dimension: ({data.count()}, {len(data.columns)})")


Dimension title_basics: (9351520, 9)
Dimension title_ratings: (1244425, 3)
Dimension title_principals: (52896017, 6)
Dimension title_episode: (7057302, 4)
Dimension name_basics: (12057828, 6)
Dimension temp_akas: (1795673, 3)

Joined title_principals to title_basics
Dimension: (9351520, 11)
Joined title_principals
Dimension: (53819205, 16)
Joined title_episode
Dimension: (53819205, 19)
Joined name_basics
Dimension: (53819205, 24)
Joined temp_akas
Dimension: (53819387, 26)


Show sample rows

In [ ]:
sample = data.rdd.takeSample(False, 10)

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.DataFrame(sample, columns =  data.columns)

,tconst,nconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,ordering,category,job,characters,parentTconst,seasonNumber,episodeNumber,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,region,language
0,tt0249306,nm0383482,tvSeries,The Naked Vicar Show,The Naked Vicar Show,0,1977,1978,55,Comedy,6.1,22,1,actor,\N,"[""Various Characters""]",None,None,None,Ross Higgins,1930,2016,"actor,soundtrack,music_department","tt0350433,tt0078637,tt0364783,tt0129666",\N,\N
1,tt8320302,nm9791474,short,Heresies,Heresies,0,2017,\N,30,"Sci-Fi,Short",None,None,1,director,\N,\N,None,None,None,Noemie Guibal,\N,\N,"assistant_director,director,writer","tt17275536,tt21438452,tt8320302",None,None
2,tt1700697,nm1422212,tvEpisode,Episode #1.1,Episode #1.1,0,2010,\N,\N,"Music,Reality-TV",None,None,8,self,\N,"[""Self""]",tt1697794,1,1,Rochelle Humes,1989,\N,actress,"tt1582196,tt1353146,tt0426721",None,None
3,tt14684714,nm0292450,video,Interview with Michael Frayn,Interview with Michael Frayn,0,2019,\N,10,"Documentary,Short",None,None,1,self,\N,"[""Self""]",None,None,None,Michael Frayn,1933,\N,"writer,actor","tt0287837,tt0820333,tt0090852,tt0097363",\N,\N
4,tt21154580,nm0134786,tvEpisode,Episode #11.74,Episode #11.74,0,2015,\N,\N,"Adventure,Drama,Romance",None,None,8,actor,\N,"[""Dr. Martos""]",tt0472642,11,74,Toni Cantó,1965,\N,"actor,soundtrack","tt1429244,tt0185125,tt0152548,tt0192877",None,None
5,tt0563599,nm0926982,tvEpisode,Rat Man,Rat Man,0,2002,\N,\N,Drama,None,None,3,actress,\N,"[""Dr. Helen Thompson""]",tt0241383,3,88,Corrinne Wicks,1968,\N,"actress,casting_department,producer","tt0241383,tt0478942,tt13678976",None,None
6,tt0437855,nm1804317,short,Beautiful People,Beautiful People,0,2003,\N,15,Short,None,None,10,composer,\N,\N,None,None,None,Phil Bambos Zacharias,\N,\N,composer,"tt8476620,tt0960060,tt0437855,tt2304617",\N,\N
7,tt14433590,nm5591614,tvEpisode,Episode #3.20,Episode #3.20,0,2021,\N,\N,Drama,None,None,7,actor,\N,"[""Buks""]",tt4015310,3,20,Gerben Kamper,\N,\N,actor,"tt0260902,tt12066016,tt13892546,tt8595478",None,None
8,tt14117910,nm9751142,tvEpisode,Episodio 5,Episodio 5,0,2017,\N,24,Documentary,None,None,1,self,\N,"[""Self - Host""]",tt8239172,3,5,J.F. Calero,\N,\N,"director,producer,writer",tt8239172,None,None
9,tt4738248,nm1159453,tvEpisode,Episode #8.97,Episode #8.97,0,2015,\N,\N,"Drama,Family",None,None,2,actress,\N,"[""Charley Bogaarts""]",tt1112587,8,97,Priscilla Knetemann,1992,\N,actress,"tt0477217,tt1112587,tt0920448,tt1282865",None,None
